This notebook is where model inferencing is done as well as experiments with hyperparameters related to beam search along with temperature, top-K and top-p sampling.

In [ ]:
!pip install transformers==4.28.0 datasets evaluate rouge-score


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=1f3152cb787f8b4627c6061b23d0d853c7d1dbc33b1042d11f0efdb6ad32ccc5
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset
import numpy as np

In [ ]:
dataset = load_dataset("mehnaazasad/arxiv_astro_co_ga")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/mehnaazasad___json/mehnaazasad--arxiv_astro_co_ga-f1b7959b2032f8d9/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
abstract = dataset['test']['abstract'][1]

In [ ]:
abstract

'  Knowing accurate Pb abundances of metal-poor stars provides constraints on\nthe Pb production mechanisms in the early Galaxy. Accurately deriving Th\nabundances permits a nucleo-chronometric age determination of the star. We\nimprove the calculation of the Pb I and Th II lines in stellar atmospheres\nbased on non-LTE line formation and evaluate the influence of departures from\nLTE on Pb and Th abundance determinations through a range of stellar\nparameters. Comprehensive model atoms for Pb I and Th II are presented. The\ndepartures from LTE lead to systematically depleted total absorption in the Pb\nI lines and positive abundance corrections. Non-LTE removes the discrepancy\nbetween the solar and the meteoritic Pb abundance. With the Holweger & Mueller\n(1974) solar model atmosphere, log eps(Pb, non-LTE) = 2.09. We revise the Pb\nand Eu abundances of the strongly r-process enhanced (r-II) stars CS 31082-001\nand HE 1523-0901 and the Roederer et al. (2010) stellar sample. Our result

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("mehnaazasad/bart-large-finetuned-arxiv-co-ga-latest")

model = AutoModelForSeq2SeqLM.from_pretrained("mehnaazasad/bart-large-finetuned-arxiv-co-ga-latest")

In [ ]:
inputs = tokenizer(abstract, return_tensors="pt").input_ids
inputs

tensor([[    0,  1437, 25225,  6030,   221,   428, 47674,  5332,     9,  4204,
            12, 29557,  2690,  1639, 16311,    15, 50118,   627,   221,   428,
           931, 14519,    11,     5,   419,  5325,     4,  5438,   710,  7223,
          1935,  9404,  2032, 50118,   873,  3194,  5332,  8621,    10, 38898,
           139,    12, 30630, 22356,  1046,  8964,     9,     5,   999,     4,
           166, 50118, 37362,     5, 21586,     9,     5,   221,   428,    38,
             8,  2032,  3082,  2301,    11, 13641, 43118, 29492, 50118,   805,
            15,   786,    12,   574,  6433,   516,  9285,     8, 10516,     5,
          2712,     9, 25624,    31, 50118,   574,  6433,    15,   221,   428,
             8,  2032, 18225, 26948,  1635,   149,    10,  1186,     9, 13641,
         50118, 46669, 20413,     4, 23539,  1421, 33310,    13,   221,   428,
            38,     8,  2032,  3082,    32,  2633,     4,    20, 50118, 17272,
          2013,  4123,    31, 20066,   483,     7, 2

In [ ]:
# Greedy search
greedy_output = model.generate(inputs, max_length=50)

In [ ]:
greedy_output

tensor([[    2,     0,     0,   510,   428,     8,  2032,  3082, 47674,  5332,
            11,     5,  4118,   579,    12,   282, 40143,   118,     9,  4204,
            12, 29557,  2690,     2]])

In [ ]:
tokenizer.decode(greedy_output[0], skip_special_tokens=True)

'Pb and Th II abundances in the solar s-nuclei of metal-poor stars'

In [ ]:
# Beam search
for beam_num in range(2, 7, 1):
  beam_output = model.generate(inputs, max_length=50, num_beams=beam_num, early_stopping=True)
  print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

Pb and Th II abundances in the solar s-nuclei of metal-poor stars
Pb and Th II abundances in the early Galaxy
Pb and Th II abundances in the solar s-nuclei of metal-poor stars
Pb I and Th II abundances in stellar atmospheres: implications for the
  solar s-nuclei
Pb I and Th II abundances in the solar s-nuclei of metal-poor stars


In [ ]:
# Use temperature to decrease the sensitivity to low probability candidates
temp_values = np.linspace(0.2, 1.0, 5)
output_arr = []
for temp_val in temp_values:
  output = model.generate(inputs, max_new_tokens=20, do_sample=True, temperature=temp_val, top_k=0)
  output_arr.append(output)

In [ ]:
output_arr

[tensor([[    2,     0,     0,     0,   510,   428,     8,  2032,  3082, 47674,
           5332,     9,  4204,  2129,   134, 35868, 41856, 37487, 13237, 10693,
              2]]),
 tensor([[    2,     0,     0,     0,   510,   428,     8,  2032,  3082, 47674,
           5332,     9,  4204,    12, 29557,  2690,    11, 13641, 43118, 29492,
              2]]),
 tensor([[    2,     0,     0,     0,   510,   428,     8,  2032,  3082, 47674,
           5332,     9,  4204,    12, 29557,  2690,    11, 13641, 43118, 29492,
              2]]),
 tensor([[    2,     0,   510,   428,     8,  2032,  3082, 47674,  5332,    11,
          13641, 43118, 29492,     2]]),
 tensor([[    2,     0,     0,   510,   428,    38,     8,  2032,  3082, 47674,
           5332,    11, 13641, 43118, 29492,    35,  8819,    13,     5, 50118,
              2]])]

In [ ]:
for idx, output in enumerate(output_arr):
  title = tokenizer.decode(output[0], skip_special_tokens=True)
  print("Temp: {0} | Title: {1}".format(temp_values[idx], title))

Temp: 0.2 | Title: Pb and Th II abundances of metal poor1745 Deaths sweatytery Prevention
Temp: 0.4 | Title: Pb and Th II abundances of metal-poor stars in stellar atmospheres
Temp: 0.6000000000000001 | Title: Pb and Th II abundances of metal-poor stars in stellar atmospheres
Temp: 0.8 | Title: Pb and Th II abundances in stellar atmospheres
Temp: 1.0 | Title: Pb I and Th II abundances in stellar atmospheres: implications for the



Based on just changing the temperature, 0.4-0.8 seems to be most human-like.

In [ ]:
# Top-K sampling
for topk in range(5, 55, 5):
  topk_output = model.generate(inputs, max_length=50, top_k=topk, do_sample=True)
  print("K: {0} | Title: {1}".format(topk, tokenizer.decode(topk_output[0], skip_special_tokens=True)))

K: 5 | Title: Pb and Th II abundances in the solar s-nuclei of metal-poor stars
K: 10 | Title: Pb and Th II abundances of metal-poor stars in the early Galaxy
K: 15 | Title: Pb I and Th II abundances of metal-poor stars in stellar atmospheres
K: 20 | Title: Pb I and Th II abundances in the solar s-nuclei
K: 25 | Title: Pb I and Th II lines in stellar atmospheres based on non-LTE line formation
K: 30 | Title: Pb and Th II abundances in the solar s-nuclei of metal-poor stars
K: 35 | Title: Pb and Th II abundances of metal-poor stars in stellar atmospheres: Implications for the
  s-process synthesis of Pb
K: 40 | Title: Pb and Th II abundances of metal-poor stars in stellar atmospheres
K: 45 | Title: Pb and Th II abundances of metal-poor stars in stellar atmospheres
K: 50 | Title: Pb and Th II abundances of metal-poor stars in stellar atmospheres


I really like 35! While it is more wordy, it gives a little more information and also specifically states Pb I instead of just Pb.

In [ ]:
# Top-p sampling
for topp in np.arange(0.8, 1.02, 0.02):
  topp_output = model.generate(inputs, max_length=50, top_k=0, top_p=topp, do_sample=True)
  print("p: {0} | Title: {1}".format(topp, tokenizer.decode(topp_output[0], skip_special_tokens=True)))

p: 0.8 | Title: Pb I and Th II abundances in the solar s-nuclei of metal-poor stars
p: 0.8200000000000001 | Title: Pb I and Th II abundances of metal-poor stars in stellar atmospheres
p: 0.8400000000000001 | Title: Pb and Th II abundances in the solar s-nuclei of metal-poor stars
p: 0.8600000000000001 | Title: Pb and Th II abundances of metal-poor stars in stellar atmospheres
p: 0.8800000000000001 | Title: Pb I and Th II abundances in stellar atmospheres based on non-LTE line formation
p: 0.9000000000000001 | Title: Pb and Th II abundances of metal-poor stars in non-LTE atmospheres
p: 0.9200000000000002 | Title: Pb and Th II abundances in the solar s-nuclei of metal-poor stars based on
  non-LTE line formation
p: 0.9400000000000002 | Title: Pb I and Th II abundances in stellar atmospheres: implications for the solar
  s-nuclei of metal-poor stars
p: 0.9600000000000002 | Title: Pb and Th II abundances of strongly r-process enhanced stars in the early
  Galaxy
p: 0.9800000000000002 | Tit

I think this was too fine of a delta because it's hard to pick out a "good" sentence. Also, need to use a more quantitative method to decide what's a good hyperparameter space.

Let's compare model performance with baseline model from earlier (First 20 in test set)

In [ ]:
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from tqdm import tqdm

In [ ]:
test_dataset = dataset['test']
abstracts = test_dataset['abstract']
ref_titles = test_dataset['title']

In [ ]:
abstracts_subset = abstracts[:20]

In [ ]:

candidate_titles = []
for i in tqdm(range(len(abstracts_subset))):
  inputs = tokenizer(abstracts_subset[i], return_tensors="pt").input_ids
  candidate = model.generate(inputs)
  candidate = tokenizer.decode(candidate[0], skip_special_tokens=True)
  candidate_titles.append(candidate)

  0%|          | 0/20 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 20/20 [03:05<00:00,  9.26s/it]


In [ ]:
from evaluate import load
metric_new = load("rouge")

In [ ]:
# ref_titles = np.array(ref_titles)[rnd_idxs]
ref_titles_subset = ref_titles[:20]
scores = metric_new.compute(predictions=candidate_titles, references=ref_titles_subset, use_stemmer=True, use_aggregator=True)
result = {key: value * 100 for key, value in scores.items()}
result = {k: round(v,4) for k, v in result.items()}

In [ ]:
result

{'rouge1': 44.7248, 'rouge2': 26.973, 'rougeL': 37.7389, 'rougeLsum': 40.1677}

Sweet! Even without any hyperparameter tuning, these scores are better than the baseline model scores below:

First 1 sentence(s): Scores {'rouge1': 33.5817, 'rouge2': 15.5748, 'rougeL': 24.6671, 'rougeLsum': 30.1371}

First 2 sentence(s): Scores {'rouge1': 25.0404, 'rouge2': 11.201, 'rougeL': 19.0928, 'rougeLsum': 23.6574}

First 3 sentence(s): Scores {'rouge1': 20.6268, 'rouge2': 10.7274, 'rougeL': 16.0666, 'rougeLsum': 19.9266}

In [ ]:
# Using a more optimum choice of hyperparameters based on tests above (these are different from the ones used in model training: num_beam=1, temperature=1, top_k=50, top_p=1)
num_beams = 5
temp = 0.7
top_k = 35
top_p = 0.94

In [ ]:

candidate_titles = []
for i in tqdm(range(len(abstracts_subset))):
  inputs = tokenizer(abstracts_subset[i], return_tensors="pt").input_ids
  output = model.generate(inputs, max_length=50,
                          num_beams=num_beams, temperature=temp,
                          top_k=top_k, top_p=top_p, do_sample=True)
  candidate = tokenizer.decode(output[0], skip_special_tokens=True)
  candidate_titles.append(candidate)

100%|██████████| 20/20 [04:25<00:00, 13.28s/it]


In [ ]:
candidate_titles

['Anisotropic cosmology with closed strings',
 'Pb and Th II abundances of metal-poor stars in stellar atmospheres',
 'Interacting dark sector with field-fluid mapping',
 'Cosmological constraints on dark matter N-body simulations',
 'Dust Extinction in Mid-Infrared Selected Luminous Quasars',
 'The Star Formation in Nearby Clouds (SFiNCs) project: X-ray and mid-infrared point sources',
 'The origin of the steep rotation curves of blue compact dwarf galaxies',
 'The VANDELS survey: the first data release',
 'Halo merging algorithm for growing voids in the Einstein de Sitter Universe',
 'Open issues in the evolution of AGN',
 'Constraining the Phantom Brane with Distance Measurements',
 'Multi-wavelength observations of blazar jets from radio to gamma-ray',
 'Chemical Abundances of Seven Stars in Moving Group W11450 (Latham 1)',
 'Quantum Field Theory and Cosmology',
 'The origin of the double red clumps in the Milky Way bulge',
 'Effects of primordial non-Gaussianity on the formation o

In [ ]:
scores = metric_new.compute(predictions=candidate_titles, references=ref_titles_subset, use_stemmer=True, use_aggregator=True)
result = {key: value * 100 for key, value in scores.items()}
result = {k: round(v,4) for k, v in result.items()}

In [ ]:
result

{'rouge1': 51.3478, 'rouge2': 30.5703, 'rougeL': 46.3357, 'rougeLsum': 48.2067}

These scores are much better than the ones from the validation set!

In [ ]:
ref_titles_subset

['String Cosmology backgrounds from Classical String Geometry',
 'Non-LTE effects on the lead and thorium abundance determinations for\n  cool stars',
 'Observational constraints and predictions of the interacting dark sector\n  with field-fluid mapping',
 'The impact of systematic uncertainties in N-body simulations on the\n  precision cosmology from galaxy clustering: a halo model approach',
 'Obscuration by Gas and Dust in Luminous Quasars',
 'Star Formation In Nearby Clouds (SFiNCs): X-ray And Infrared Source\n  Catalogs And Membership',
 'Formation and evolution of blue compact dwarfs: The origin of their\n  steep rotation curves',
 'The VANDELS ESO public spectroscopic survey: observations and first data\n  release',
 'Merging Tree Algorithm of Growing Voids in Self Similar and CDM Models',
 'Demography of obscured and unobscured AGN: prospects for a Wide Field\n  X-ray Telescope',
 'Constraining the Cosmology of the Phantom Brane using Distance Measures',
 'Synchrotron and inver